# Monte Carlo Simulation for FE
## IEOR 4703

### Calculting Greeks via simulation (Black-Merton-Scholes)

1. w/o common random numbers
2. w/ common random numbers


In [1]:
import numpy as np
from scipy.stats import norm

from BMS import BMS_price, BMS_delta

In [2]:
# parameters
s0 = 1000
K = 1010
sig = 0.35
T = 1
r = 0.04
q = 0.015

C = BMS_price('call', s0, K, r, q, sig, T)
print('C = ' + str(C))

#delS = 0.005
#delSig = 0.0005

nN = 1000000

delS   = np.array([50.0, 10.0, 1.0, 0.5, 0.25, 0.10, 0.05, 0.01, 0.005, 0.001])
len1 = len(delS)

delSig = np.array([0.05, 0.02, 0.01, 0.005, 0.001, 0.0005, 0.0001])
len2 = len(delSig)

C = 143.31160604446944


### Exact values for $\Delta$, $\Gamma$, and $vega$

In [3]:
d1 = (np.log(s0/K) + (r - q + sig**2/2)*T)/(sig*np.sqrt(T))
d2 = d1 - sig*np.sqrt(T)

delC =  np.exp(-q*T) * norm.cdf(d1)
# delP = -np.exp(-q*T)* norm.cdf(-d1)

gammaC = np.exp(-q*T)*norm.pdf(d1)/(s0*sig*np.sqrt(T))
# gammaP = gammaC

vegaC = s0*np.sqrt(T)*np.exp(-q*T)*norm.pdf(d1)/100
# vegaP = vegaC

### Simulation

In [4]:
#np.random.seed(94154573)
np.random.seed(64154573)

In [5]:
z1 = np.random.randn(nN)
z2 = np.random.randn(nN)
z3 = np.random.randn(nN)

### $\Delta$ & $\Gamma$

In [6]:
delC_CRN1 = np.zeros(len1)
delC_CRN2 = np.zeros(len1)
gam_CRN   = np.zeros(len1)

delC_tilde1 = np.zeros(len1)
delC_tilde2 = np.zeros(len1)
gam_tilde   = np.zeros(len1)

for j in range(len1):

    tmp1U = (s0 + delS[j])*np.exp((r-q-sig**2/2)*T)
    tmp1  =  s0      *np.exp((r-q-sig**2/2)*T)
    tmp1D = (s0 - delS[j])*np.exp((r-q-sig**2/2)*T)

    tmp2 = sig*np.sqrt(T)

    # ---------------------------------
    # (1) W/ Common Random Numbers
    s = tmp1*np.exp(tmp2*z1)
    tmp = np.exp(-r*T)*np.maximum(s-K,0)

    s = tmp1U*np.exp(tmp2*z1)
    tmpU = np.exp(-r*T)*np.maximum(s-K,0)

    s = tmp1D*np.exp(tmp2*z1)
    tmpD = np.exp(-r*T)*np.maximum(s-K,0)

    delC_CRN1[j] = np.mean((tmpU - tmp))/delS[j]
    delC_CRN2[j] = np.mean((tmpU - tmpD))/(2*delS[j])

    gam_CRN[j]  = np.mean((tmpU - 2*tmp + tmpD))/(delS[j]**2)

    # ----------------------------------
    # (2) W/O Common Random Numbers
    s =  tmp1*np.exp(tmp2*z1)
    tmp = np.exp(-r*T)*np.maximum(s - K,0)

    s = tmp1U*np.exp(tmp2*z2)
    tmpU = np.exp(-r*T)*np.maximum(s - K,0)

    s = tmp1D*np.exp(tmp2*z3)
    tmpD = np.exp(-r*T)*np.maximum(s - K,0)

    delC_tilde1[j] = np.mean((tmpU - tmp))/delS[j]
    delC_tilde2[j] = np.mean((tmpU - tmpD))/(2*delS[j])

    gam_tilde[j]  = np.mean((tmpU - 2*tmp + tmpD))/(delS[j]**2)



### $Vega$

In [7]:
vega_CRN   = np.zeros(len2)
vega_tilde = np.zeros(len2)

for j in range(len2):

    tmp1U  =  s0 * np.exp((r - q - (sig + delSig[j])**2/2)*T)
    tmp1D  =  s0 * np.exp((r - q - (sig - delSig[j])**2/2)*T)

    tmp2U = (sig + delSig[j]) * np.sqrt(T)
    tmp2D = (sig - delSig[j]) * np.sqrt(T)

    # (1) W/ Common Random Numbers
    s = tmp1U * np.exp(tmp2U*z1)
    tmpU = np.exp(-r*T) * np.maximum(s-K,0)

    s = tmp1D * np.exp(tmp2D*z1)
    tmpD = np.exp(-r*T) * np.maximum(s-K,0)

    vega_CRN[j] = np.mean((tmpU - tmpD)/(2*delSig[j]))/100

    # (2) W/O Common Random Numbers
    s = tmp1U * np.exp(tmp2U*z1)
    tmpU = np.exp(-r*T) * np.maximum(s-K,0)

    s = tmp1D * np.exp(tmp2D*z2)
    tmpD = np.exp(-r*T) * np.maximum(s-K,0)

    vega_tilde[j] = np.mean((tmpU - tmpD)/(2*delSig[j]))/100


### Displaying results

In [8]:
print('==================')
print('\tDelta')
print('==================')
print('Exact: %f' % delC)
print('W/  CRN:')
print(delC_CRN1)
print('W/O CRN:')
print(delC_tilde1)

print('==================')
print('\tGamma')
print('==================')
print('Exact: %f' % gammaC)
print('W/  CRN:')
print(gam_CRN)
print('W/O CRN:')
print(gam_tilde)

print('==================')
print('\tVega')
print('==================')
print('Exact: %f' % vegaC)
print('W/  CRN:')
print(vega_CRN)
print('W/O CRN:')
print(vega_tilde)

	Delta
Exact: 0.577556
W/  CRN:
[0.60423327 0.58319945 0.57822665 0.57795764 0.57781679 0.57773945
 0.57771753 0.57769703 0.57769283 0.57769044]
W/O CRN:
[  0.60276092   0.57407997   0.48662874   0.3946543    0.21112985
  -0.33914137  -1.25614688  -8.59197365 -17.76173378 -91.11979107]
	Gamma
Exact: 0.001096
W/  CRN:
[0.00109386 0.00110414 0.00107949 0.00107631 0.00101018 0.00097319
 0.00098879 0.00133438 0.00138838 0.0010667 ]
W/O CRN:
[ 8.80791329e-04 -4.78189557e-03 -5.97204850e-01 -2.39453621e+00
 -9.58625335e+00 -5.99384572e+01 -2.39782104e+02 -5.99507996e+03
 -2.39805744e+04 -5.99519395e+05]
	Vega
Exact: 3.837744
W/  CRN:
[3.84739797 3.84778566 3.84786563 3.84788467 3.84788141 3.84788116
 3.84788028]
W/O CRN:
[3.85378663 3.86770371 3.89061332 3.93644507 4.30320177 4.76169298
 8.4296013 ]
